In [1]:
# Does not need to be executed if
# ~/.ipython/profile_default/ipython_config.py
# exists and contains:
# c.InteractiveShell.ast_node_interactivity = 'all'

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

In [40]:
# Written by *** for COMP9021
#
# Uses data available at http://data.worldbank.org/indicator
# on Forest area (% of land area) and Agricultural land area (% of land area).
#
# Prompts the user for two distinct years between 1960 and 2021,
# separated by two consecutive hyphens with possibly spaces before
# and after, as well as a strictly positive integer N,
# and outputs the top N countries where:
# - agricultural land area (as a ratio) has strictly increased from
#   oldest input year to most recent input year;
# - forest area (as a ratio) has strictly increased from oldest input
#   year to most recent input year;
# - the ratio of increase in agricultural land area to
#   increase in forest area (so a ratio of ratio differences)
#   determines output order.
#
# Countries are output from those whose ratio of increases is largest
# to those whose ratio of increases is smallest, as a floating point
# number with 2 digits after the decimal point.
#
# In the unlikely case where many countries share the same ratio
# of increases, countries are output in lexicographic order.
#
# In case fewer than N countries are found, only that number of
# countries is output.
#
# If no country has data for both years, then a special message is output.
#
# The data directory is stored in the working directory.


from collections import defaultdict
import csv
from pathlib import Path
import sys




# 1. Get Inputs
def get_years():    
    user_input = input('Input two distinct years in the range 1960 -- 2021: ')
    user_input = user_input.replace(" ", "")
    if user_input.find('--') != -1 and user_input.find('---') == -1:
        years = user_input.split('--')
        if len(years) == 2 and len(years[0]) == 4 and len(years[1]) == 4 and years[0].isnumeric() and years[1].isnumeric():
            if not (int(years[0]) < 1960 or int(years[0]) > 2021 or int(years[1]) < 1960 or int(years[1]) > 2021 or years[0] == years[1]):
                return min(years[0], years[1]), max(years[0], years[1])
    return (-1,-1)

# 1.1 Get Inputs
def get_num_countries():
    n = input('Input a strictly positive integer: ')
    if n.isnumeric() and int(n) > 0:
        return n
    return -1

# 3. Function to read data files and return a dictionary of data
def read_input_files(input_file):    
    data = []
    with open(input_file) as file:
        lines = csv.reader(file)
        next(lines)
        next(lines)
        next(lines)
        next(lines)
        headers = next(lines)
        for line in lines:
            y1_data = line[headers.index(y1)]
            y2_data = line[headers.index(y2)]
            if y1_data == "":
                y1_data = '0'
            if y2_data == "":
                y2_data = '0'
            data.append({'Country Name':line[0], \
                              y1:float(y1_data), \
                              y2:float(y2_data)})
    return data

# 4. Define function to calculate the ration
def calc_ratio(a_y1, a_y2, f_y1, f_y2):
    try:
        result = (a_y2-a_y1) / (f_y2-f_y1)
    except ZeroDivisionError:
        result = 0
    return result

# 5. Create a dictionary of countries with their ratios
def calculate_all_ratios(forest_data, agriculture_data):
    ratio_data = {}
    for i in range(len(forest_data)):
        f1 = forest_data[i][y1]
        f2 = forest_data[i][y2]
        a1 = agriculture_data[i][y1]
        a2 = agriculture_data[i][y2]
        if a2 < a1 or f2 < f1:
            continue
        ratio = calc_ratio(a1, a2, f1, f2)
        if ratio >= 0:
            ratio_data.update({forest_data[i]["Country Name"]:ratio})
    return sorted(ratio_data.items(), key=lambda item: item[1])


# 6. Print final output
def print_final_output(n, sorted_ratio_data):
    # TODO: if there are fewer than n countries present, then only print those number of countries
    n = min(int(n), len(sorted_ratio_data))
    if n == '1':
        print(f'Here is the top country or category where, between {y1} and {y2},')
    else:
        print(f'Here are the top {n} countries or categories where, between {y1} and {y2},')
    print('  the ratios of agricultural land area and forest area')
    print('  (over total land) have both strictly increased,')
    print('  listed from the countries where the ratio of increases')
    print('  is largest, to those where it is smallest:')
    for i in range(1, int(n) + 1, ):
        print(f'{sorted_ratio_data[-i][0]} ({sorted_ratio_data[-i][1]:.2f})')
    

# 1. Get Inputs
y1, y2 = get_years()
if (y1, y2) != (-1, -1):
    n = get_num_countries()
    if int(y1) > 1989:
        if n != -1:
            # print(f'Range = {y1} -- {y2}, n = {n}')

            # 2. Define the files to be read
            forest_file = Path('Areas/Forest/API_AG.LND.FRST.ZS_DS2_en_csv_v2_4901640.csv')
            agricultural_land_file = Path('Areas/Agricultural Land/API_AG.LND.AGRI.ZS_DS2_en_csv_v2_4902513.csv')

            # 3. Function to read data files and return a dictionary of data
            agriculture_data = read_input_files(agricultural_land_file)
            forest_data = read_input_files(forest_file)

            # 5. Create a dictionary of countries with their ratios
            sorted_ratio_data = calculate_all_ratios(forest_data, agriculture_data)


            # 6. Print final output
            print_final_output(n, sorted_ratio_data)
        else:
            print("Fail I guess?")
    else:
        print("I do not have data for any country for at least one of those years.")
else:
    print("Not a valid range of years, giving up...")

Input two distinct years in the range 1960 -- 2021: 2000--2005
Input a strictly positive integer: 100


AttributeError: 'list' object has no attribute 'update'

# TODO
- if ratios are same, then sort by lexigraphic?
